In [1]:
from pathlib import Path
import pyaudio
from pydub import AudioSegment
import numpy
from scipy import signal, fftpack
# import matplotlib.pyplot as plt

In [2]:
working_dir = Path('e:/data/kaudio')
test_file = working_dir / '5 Luda.mp4.aac'
test_dir = working_dir / 'test'
test_export = test_dir / "test.mp3"
pa = pyaudio.PyAudio()

In [4]:
pa.get_default_host_api_info()

{'index': 0,
 'structVersion': 1,
 'type': 2,
 'name': 'MME',
 'deviceCount': 12,
 'defaultInputDevice': 1,
 'defaultOutputDevice': 7}

In [5]:
pa.get_default_output_device_info()

{'index': 7,
 'structVersion': 2,
 'name': 'VoiceMeeter Input (VB-Audio Voi',
 'hostApi': 0,
 'maxInputChannels': 0,
 'maxOutputChannels': 8,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [6]:
pa.get_default_input_device_info()

{'index': 1,
 'structVersion': 2,
 'name': 'Line In (Realtek High Definitio',
 'hostApi': 0,
 'maxInputChannels': 2,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [7]:
for id in range(pa.get_host_api_count()):
    print(pa.get_host_api_info_by_index(id))

{'index': 0, 'structVersion': 1, 'type': 2, 'name': 'MME', 'deviceCount': 12, 'defaultInputDevice': 1, 'defaultOutputDevice': 7}
{'index': 1, 'structVersion': 1, 'type': 1, 'name': 'Windows DirectSound', 'deviceCount': 12, 'defaultInputDevice': 12, 'defaultOutputDevice': 18}
{'index': 2, 'structVersion': 1, 'type': 3, 'name': 'ASIO', 'deviceCount': 6, 'defaultInputDevice': 24, 'defaultOutputDevice': 24}
{'index': 3, 'structVersion': 1, 'type': 13, 'name': 'Windows WASAPI', 'deviceCount': 10, 'defaultInputDevice': 35, 'defaultOutputDevice': 33}
{'index': 4, 'structVersion': 1, 'type': 11, 'name': 'Windows WDM-KS', 'deviceCount': 24, 'defaultInputDevice': 46, 'defaultOutputDevice': 57}


In [14]:
pa.get_host_api_info_by_type(pyaudio.paWASAPI)

{'index': 3,
 'structVersion': 1,
 'type': 13,
 'name': 'Windows WASAPI',
 'deviceCount': 10,
 'defaultInputDevice': 35,
 'defaultOutputDevice': 33}

In [31]:
pa.get_device_info_by_host_api_device_index(3, 0)

{'index': 30,
 'structVersion': 2,
 'name': 'VoiceMeeter VAIO3 Input (VB-Audio VoiceMeeter VAIO3)',
 'hostApi': 3,
 'maxInputChannels': 0,
 'maxOutputChannels': 2,
 'defaultLowInputLatency': 0.0,
 'defaultLowOutputLatency': 0.003,
 'defaultHighInputLatency': 0.0,
 'defaultHighOutputLatency': 0.01,
 'defaultSampleRate': 48000.0}

In [12]:
for id in range(pa.get_device_count()):
  dev_dict = pa.get_device_info_by_index(id)

  if 'hostApi' in dev_dict and dev_dict['hostApi'] == 3:
      for key, value in dev_dict.items():
          print(key, value)
      print('\n')

index 30
structVersion 2
name VoiceMeeter VAIO3 Input (VB-Audio VoiceMeeter VAIO3)
hostApi 3
maxInputChannels 0
maxOutputChannels 2
defaultLowInputLatency 0.0
defaultLowOutputLatency 0.003
defaultHighInputLatency 0.0
defaultHighOutputLatency 0.01
defaultSampleRate 48000.0


index 31
structVersion 2
name Front Output (Realtek High Definition Audio)
hostApi 3
maxInputChannels 0
maxOutputChannels 2
defaultLowInputLatency 0.0
defaultLowOutputLatency 0.003
defaultHighInputLatency 0.0
defaultHighOutputLatency 0.01
defaultSampleRate 48000.0


index 32
structVersion 2
name VoiceMeeter Aux Input (VB-Audio VoiceMeeter AUX VAIO)
hostApi 3
maxInputChannels 0
maxOutputChannels 2
defaultLowInputLatency 0.0
defaultLowOutputLatency 0.003
defaultHighInputLatency 0.0
defaultHighOutputLatency 0.01
defaultSampleRate 48000.0


index 33
structVersion 2
name VoiceMeeter Input (VB-Audio VoiceMeeter VAIO)
hostApi 3
maxInputChannels 0
maxOutputChannels 2
defaultLowInputLatency 0.0
defaultLowOutputLatency 0.003


In [4]:
test_as = AudioSegment.from_file(test_file, test_file.suffix[1:])
print("Sample rate: {}".format(test_as.frame_rate))
print("Number of bits: {}-bit".format(test_as.sample_width * 8))
print("Number of channels: {}".format(test_as.channels))

Sample rate: 48000
Number of bits: 16-bit
Number of channels: 1


In [30]:
stream = pa.open(
    rate=48000,     # sampling rate
    channels=2, # number of output channels
    format=pyaudio.paInt16,  # sample format and length
    output=True,             # output stream flag
    output_device_index=9,   # output device index
    input=True,                   # input stream flag
    input_device_index=3,         # input device index maybe 13
    frames_per_buffer=1024,  # buffer length
)

# stream_out.write(test_as.get_array_of_samples().tobytes())

# read 5 seconds of the input stream
# input_audio = stream_in.read(5 * 48000)

In [51]:
a = stream.read(1024)

In [54]:
stream.write(a)

In [33]:
len(a)

4096

In [34]:
# interleaved channels
frame = numpy.frombuffer(a, dtype=numpy.int16)
frame = numpy.stack((frame[::2], frame[1::2]), axis=0)  # channels on separate axes

In [40]:
frame.shape

(2, 1024)

In [45]:
frame[0].shape

(1024,)

In [14]:
b

array([   0,    0,    0, ..., -256,  255,    0], dtype=int16)

In [ ]:
# len(test_as.get_array_of_samples())/48000
len(test_as.get_array_of_samples())
# test_as.export(test_export)
# test_as.array_type

In [ ]:
samples = test_as.get_array_of_samples()


In [ ]:
samples = numpy.array(samples)

In [ ]:
samples.shape

In [ ]:
f, t, sxx = signal.spectrogram(samples, 1024)

In [ ]:
import pyaudio
# import wave

CHUNK = 1024
# FORMAT = pyaudio.paInt24
# CHANNELS = 2
# RATE = 44100
# RECORD_SECONDS = 5
# WAVE_OUTPUT_FILENAME = "output.wav"

# p = pyaudio.PyAudio()

# stream = p.open(format=FORMAT,
#                 channels=CHANNELS,
#                 rate=RATE,
#                 input=True,
#                 frames_per_buffer=CHUNK)

print("* recording")

# frames = []
# frames.append(data)



# for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
try:
    stream_in.start_stream()
    while True:
        stream_out.write(stream_in.read(CHUNK))
        # stream_in.read(CHUNK)
        
except:
    pass
print("* done recording")
stream_in.stop_stream()
stream_in.close()
pa.terminate()

# wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
# wf.setnchannels(CHANNELS)
# wf.setsampwidth(p.get_sample_size(FORMAT))
# wf.setframerate(RATE)
# wf.writeframes(b''.join(frames))
# wf.close()

In [29]:
stream.close()


In [34]:
pa.terminate()

In [2]:
import numpy as np
class numpy_data_buffer:
    """
    A fast, circular FIFO buffer in numpy with minimal memory interactions by using an array of index pointers
    """

    def __init__(self, n_windows, samples_per_window, dtype = np.float32, start_value = 0, data_dimensions = 1):
        self.n_windows = n_windows
        self.data_dimensions = data_dimensions
        self.samples_per_window = samples_per_window
        self.data = start_value * np.ones((self.n_windows, self.samples_per_window), dtype = dtype)

        if self.data_dimensions == 1:
            self.total_samples = self.n_windows * self.samples_per_window
        else:
            self.total_samples = self.n_windows

        self.elements_in_buffer = 0
        self.overwrite_index = 0

        self.indices = np.arange(self.n_windows, dtype=np.int32)
        self.last_window_id = np.max(self.indices)
        self.index_order = np.argsort(self.indices)

    def append_data(self, data_window):
        self.data[self.overwrite_index, :] = data_window

        self.last_window_id += 1
        self.indices[self.overwrite_index] = self.last_window_id
        self.index_order = np.argsort(self.indices)

        self.overwrite_index += 1
        self.overwrite_index = self.overwrite_index % self.n_windows

        self.elements_in_buffer += 1
        self.elements_in_buffer = min(self.n_windows, self.elements_in_buffer)

    def get_most_recent(self, window_size):
        ordered_dataframe = self.data[self.index_order]
        if self.data_dimensions == 1:
            ordered_dataframe = np.hstack(ordered_dataframe)
        return ordered_dataframe[self.total_samples - window_size:]

    def get_buffer_data(self):
        return self.data[:self.elements_in_buffer]

In [4]:
asdf = numpy_data_buffer(1000//2, 48000//1000)

In [9]:
asdf.data.shape

(500, 48)

In [10]:
asdf.total_samples

24000

In [13]:
asdf.indices

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [12]:
asdf.indices.shape

(500,)

In [14]:
asdf.index_order

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [6]:
asdf.get_buffer_data()

array([], shape=(0, 48), dtype=float32)

In [17]:
asdf.overwrite_index

0

In [30]:
asdf.data[:500].shape

(500, 48)

In [18]:
asdf.data[asdf.overwrite_index, :].shape

(48,)

In [22]:
500 % asdf.n_windows

0

In [32]:
max(asdf.indices)

499

In [39]:
asdf.indices

array([499,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [46]:
asdf.indices[0] = 500

In [40]:
asdf.index_order

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [47]:
index = np.argsort(asdf.indices)

In [48]:
index

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [44]:
asdf.data[index]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [50]:
np.hstack(asdf.data[index])

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)